In [87]:
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [ ]:
#tool create
#pehla tool conversion factor ke liye api ko hit krega aur conversion rate btayega
#dusra tool multiply karne ke liye
@tool
def get_conversion_factor(base_currency : str, target_currency : str)->float:
    """
        This func fetches the currency conversion factor between base and target currency
    """
    url =  f'https://v6.exchangerate-api.com/v6/079907485247991700a59609/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency_values : int, conversion_rate : Annotated[float,InjectedToolArg])->float: # llm donot try to fill this arg it means that i will inject this value after running earlier tools
    """
        given a currency conversion rate this function calculates the target currency from a given base currency value
    """
    return base_currency_values * conversion_rate

In [89]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1771372802,
 'time_last_update_utc': 'Wed, 18 Feb 2026 00:00:02 +0000',
 'time_next_update_unix': 1771459202,
 'time_next_update_utc': 'Thu, 19 Feb 2026 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.6945}

In [90]:
convert.invoke({'base_currency_values':10,'conversion_rate':90.6945})

906.945

In [91]:
# tool binding
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

In [92]:
llm_with_tools = llm.bind_tools([get_conversion_factor,convert])

In [93]:
# tool calling
messages = [HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR')]


In [94]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [95]:
ai_message = llm_with_tools.invoke(messages)

In [96]:
messages.append(ai_message)

In [97]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'c12034be-3692-4635-bbdf-43f5f17c5baa',
  'type': 'tool_call'}]

In [103]:
while True:
    ai_message = llm_with_tools.invoke(messages)
    messages.append(ai_message)

    if not ai_message.tool_calls:
        break

    for tool_call in ai_message.tool_calls:
        if tool_call["name"] == "get_conversion_factor":
            tool_msg = get_conversion_factor.invoke(tool_call)

        elif tool_call["name"] == "convert":
            tool_call["args"]["conversion_rate"] = conversion_rate
            tool_msg = convert.invoke(tool_call)

        messages.append(tool_msg)


In [104]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"base_currency": "USD", "target_currency": "INR"}'}, '__gemini_function_call_thought_signatures__': {'c12034be-3692-4635-bbdf-43f5f17c5baa': 'CocLAb4+9vtCreVvkqJJ2uwnkeD2zNzNE/my+EJGqH4ZSYzMrtDWmUMJ8klnTcOMO+z7Hi1b9mKxeWNcLD2q5WqxNpCCtzliXrkuh0QrFNGfSE9VbPGaKq5X4LFMikIo1CcREtOEe3MBvRolEhdK+c5t0UAlH1xaGsI2h0vCPgr0IuhN/numM8kvGHmSzZCLDbpmBlVS33R/l/eRh+Z302Zs3eKzDOsypEk9ES7exqjZVZsi1nQbZYvIVSBhzXVuN/RyMepvFFUcKy7BWr8IXi3Gp/GoJtkUKHL7q3i/1QSk1WIOVxtKqUDXpGEzuXAnvs2HyPg35AuwVo4HSK7tSYW+u5Fdg+3T4eztS3iVWARtF9EUHVSGM8Y1+mO3u+px/sjXK7FJjvQ9TBJBBUa3AVNJ9D9qi1Oh77EF6XJbtmYtw9g1MCep61Ny97+j31rDgMm9/ZsZDxZQjIqFhJMvDPjUBYH/UzDD+xC3gFQyOQHU2+JfMSY18U1z86rJcgBcWIiSkDFo0ajfgUQZb9+x+Hfykr39am+B3CzRot72F48kTKVX/J9dfnAFyO8TEEK9FZwM3

In [106]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c7018-f89e-7720-a65a-639b1e7bea2d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 456, 'output_tokens': 0, 'total_tokens': 456, 'input_token_details': {'cache_read': 0}})

In [ ]:
print(messages[-1].content)
